In [428]:
import json
import requests

import pandas as pd
from pandas.io.json import json_normalize
import datetime,time
from datetime import datetime
from math import atan2,degrees
import numpy as np

import matplotlib.pyplot as plt

In [429]:
#for animation
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML

In [430]:
url='https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv'

In [431]:
url2= 'https://api.covid19india.org/csv/latest/raw_data3.csv'

In [432]:
url1='https://api.covid19india.org/csv/latest/raw_data.csv'

In [433]:
df=pd.read_csv(url)

In [434]:
df.head()

,Updated On,State,Total Tested,Tag (Total Tested),Positive,Negative,Unconfirmed,Cumulative People In Quarantine,Total People Currently in Quarantine,Tag (People in Quarantine),...,Corona Enquiry Calls,Num Calls State Helpline,Source1,Unnamed: 17,Source2,Test positivity rate,Tests per thousand,Tests per million,Tests per positive case,Population NCP 2019 Projection
0,17/04/2020,Andaman and Nicobar Islands,1403.0,Samples Sent,12.0,1210.0,181.0,NaN,NaN,NaN,...,NaN,NaN,https://t.me/indiacovid/2550,NaN,NaN,0.86%,3.53,3534.0,117.0,397000.0
1,24/04/2020,Andaman and Nicobar Islands,2679.0,Samples Sent,27.0,NaN,246.0,NaN,614.0,NaN,...,NaN,280.0,https://t.me/indiacovid/3147?single,NaN,NaN,1.01%,6.75,6748.0,99.0,397000.0
2,27/04/2020,Andaman and Nicobar Islands,2848.0,Samples Sent,33.0,NaN,106.0,NaN,724.0,NaN,...,NaN,298.0,https://t.me/indiacovid/3365?single,NaN,NaN,1.16%,7.17,7174.0,86.0,397000.0
3,01/05/2020,Andaman and Nicobar Islands,3754.0,Samples Sent,33.0,NaN,199.0,NaN,643.0,Institutional,...,NaN,340.0,https://t.me/indiacovid/3781,NaN,NaN,0.88%,9.46,9456.0,114.0,397000.0
4,16/05/2020,Andaman and Nicobar Islands,6677.0,Samples Sent,33.0,NaN,136.0,NaN,16.0,Institutional,...,NaN,471.0,https://t.me/indiacovid/4925,NaN,NaN,0.49%,16.82,16819.0,202.0,397000.0


In [435]:
df=df.fillna(0)

In [436]:
df['Test positivity rate']=df['Test positivity rate'].str.rstrip('%').astype('float') 

In [437]:
top = ['Andhra Pradesh','Bihar', 'Delhi','Gujarat','Karnataka', 'Kerala', 
       'Madhya Pradesh', 'Maharashtra', 'Punjab', 'Rajasthan','Tamil Nadu', 'Telangana', 'Uttar Pradesh','West Bengal']

In [438]:
today = datetime.today().strftime('%b-%d')

In [439]:
pts=df.loc[df['State'].isin(top)]
pts['Updated On']= pd.to_datetime(pts['Updated On'], dayfirst = True)
pts['Positive Rate']=pts['Positive']/pts['Total Tested']*100

C:\Users\Siva\Anaconda3\envs\torch1.3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Siva\Anaconda3\envs\torch1.3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [440]:
pts=pts[['Updated On','State','Positive','Positive Rate']]

In [441]:
pts['Daily']=pts.groupby('State')[['State', 'Positive']].diff(periods=1)

In [442]:
pts.drop(pts[pts['Daily']<1].index, inplace = True)

In [443]:
pts.drop(pts[pts['Daily']==4122].index, inplace = True)
pts.drop(pts[pts['Daily']==3173].index, inplace = True)
pts.drop(pts[pts['Daily']==2230].index, inplace = True)

In [444]:
pts.fillna(pts.mean(), inplace = True)

In [445]:
import plotly.express as px

color_discrete_map={'Bihar': '#fe0a11', 'Gujarat': '#f0b884', 'Karnataka': '#ddb0ff',
                    'Kerala':'#a4cc8b', 'Madhya Pradesh':'#4287f5', 'Maharashtra':'#f7baeb', 'Punjab':'#d0bb5f', 
                    'Rajasthan':'#5edef7','Tamil Nadu':'#c48381', 'Telangana':'#a0b34f', 'Uttar Pradesh':'#42fb50', 
                    'West Bengal':'#b0a1f5',}
fig = px.scatter(pts, x='Updated On', y='Positive Rate',color="State",
                 size='Daily',width=900, height=500,size_max=60,color_discrete_map=color_discrete_map)
fig.update_traces(mode='markers',  marker=dict(sizemode='area',sizeref=3, line_width=1, opacity= 0.5,))

fig.update_layout(
    title=dict(
        text = 'COVID-19: Positivity Rate vs Daily Cases 2020-'+today,
        xanchor= 'center',
        y=0.95,
        x = 0.5),
    legend=dict(title='States'),
    xaxis=dict(
        title='Tested On Date',
        gridcolor='white',
        gridwidth=2,
    ),
    yaxis=dict(
        title='Positivity  Rate %',
        gridcolor='white',
        gridwidth=2,
    ),
    annotations=[
            dict(
            x=1.2,
            y=-0.20,
            showarrow=False,
            text="by @sivakd , data @covid19indiaorg",
            xref="paper",
            yref="paper",
            font = dict(color = "gray") ),
            ],
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

for p in anon['Positive Rate']:
    overlap1 = anon[anon['Positive Rate']==p]['State']=='Karnataka'
    overlap2 = anon[anon['Positive Rate']==p]['State']=='Karnataka'
    #fig.add_annotation(x='2020-05-20',y=p,font = dict(size = 10),
    #               text=anon[anon['Positive Rate']==p]['State'].iloc[0],showarrow=True,xref='x',yref='y',align='left',)

fig.write_image("Comparison-Positive-Rate-vs-Cases-Growth-2020-"+today+".png")
fig.show()

In [446]:
if(p>4):
    fig.add_annotation(x='2020-05-20',y=p,showarrow=False,font = dict(size = 10),
                   text=anon[anon['Positive Rate']==p]['State'].iloc[0],xref='x',yref='y',)
elif((p<4) and (p>3)):
    fig.add_annotation(x='2020-05-21',y=p,showarrow=False,font = dict(size = 10),
               text=anon[anon['Positive Rate']==p]['State'].iloc[0])
elif(overlap1.all() or overlap2.all()):
    fig.add_annotation(x='2020-05-15',y=p,showarrow=True,font = dict(size = 10),
               text=anon[anon['Positive Rate']==p]['State'].iloc[0],ax=20,)
else:
    fig.add_annotation(x='2020-05-21',y=p,showarrow=False,font = dict(size = 10),
           text=anon[anon['Positive Rate']==p]['State'].iloc[0],ax=20,)
